In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# load data
data = pd.read_csv('tech_60m.csv')
data.head()
data.shape

,moddate,AAPL_VWP_t_14,MSFT_VWP_t_14,FB_VWP_t_14,TWTR_VWP_t_14,AAPL_VWP_t_13,MSFT_VWP_t_13,FB_VWP_t_13,TWTR_VWP_t_13,AAPL_VWP_t_12,...,FB_VWP_t_2,TWTR_VWP_t_2,AAPL_VWP_t_1,MSFT_VWP_t_1,FB_VWP_t_1,TWTR_VWP_t_1,AAPL_VWP,MSFT_VWP,FB_VWP,TWTR_VWP
0,2017-01-03 18:00:00,0.008251,0.009330,0.007926,0.009827,0.008258,0.009330,0.007946,0.009876,0.008258,...,0.007981,0.009851,0.008257,0.009302,0.007980,0.009847,0.008254,0.009279,0.007977,0.009839
1,2017-01-03 19:00:00,0.008258,0.009330,0.007946,0.009877,0.008258,0.009333,0.007939,0.009870,0.008267,...,0.007980,0.009847,0.008254,0.009280,0.007978,0.009839,0.008253,0.009280,0.007972,0.009838
2,2017-01-03 20:00:00,0.008258,0.009334,0.007939,0.009871,0.008267,0.009351,0.007955,0.009848,0.008273,...,0.007978,0.009840,0.008253,0.009281,0.007973,0.009838,0.008257,0.009283,0.007969,0.009829
3,2017-01-03 21:00:00,0.008267,0.009351,0.007956,0.009848,0.008273,0.009342,0.007944,0.009854,0.008267,...,0.007973,0.009839,0.008257,0.009283,0.007969,0.009829,0.008259,0.009287,0.007976,0.009831
4,2017-01-03 22:00:00,0.008273,0.009342,0.007944,0.009855,0.008267,0.009313,0.007932,0.009803,0.008227,...,0.007970,0.009830,0.008260,0.009288,0.007976,0.009832,0.008262,0.009292,0.007982,0.009834


(8642, 61)

In [3]:
# get features and convert to array so that we can feed into model
data_x = data.iloc[:, 1:57]
data_x.head()
data_x.shape
data_x = data_x.values
data_x

,AAPL_VWP_t_14,MSFT_VWP_t_14,FB_VWP_t_14,TWTR_VWP_t_14,AAPL_VWP_t_13,MSFT_VWP_t_13,FB_VWP_t_13,TWTR_VWP_t_13,AAPL_VWP_t_12,MSFT_VWP_t_12,...,FB_VWP_t_3,TWTR_VWP_t_3,AAPL_VWP_t_2,MSFT_VWP_t_2,FB_VWP_t_2,TWTR_VWP_t_2,AAPL_VWP_t_1,MSFT_VWP_t_1,FB_VWP_t_1,TWTR_VWP_t_1
0,0.008251,0.009330,0.007926,0.009827,0.008258,0.009330,0.007946,0.009876,0.008258,0.009333,...,0.007962,0.009804,0.008263,0.009306,0.007981,0.009851,0.008257,0.009302,0.007980,0.009847
1,0.008258,0.009330,0.007946,0.009877,0.008258,0.009333,0.007939,0.009870,0.008267,0.009350,...,0.007981,0.009851,0.008258,0.009303,0.007980,0.009847,0.008254,0.009280,0.007978,0.009839
2,0.008258,0.009334,0.007939,0.009871,0.008267,0.009351,0.007955,0.009848,0.008273,0.009342,...,0.007980,0.009848,0.008255,0.009280,0.007978,0.009840,0.008253,0.009281,0.007973,0.009838
3,0.008267,0.009351,0.007956,0.009848,0.008273,0.009342,0.007944,0.009854,0.008267,0.009313,...,0.007978,0.009840,0.008254,0.009281,0.007973,0.009839,0.008257,0.009283,0.007969,0.009829
4,0.008273,0.009342,0.007944,0.009855,0.008267,0.009313,0.007932,0.009803,0.008227,0.009312,...,0.007973,0.009839,0.008257,0.009284,0.007970,0.009830,0.008260,0.009288,0.007976,0.009832


(8642, 56)

array([[0.00825107, 0.00933008, 0.00792621, ..., 0.00930246, 0.00797956,
        0.00984677],
       [0.0082581 , 0.00933033, 0.00794631, ..., 0.00927969, 0.00797767,
        0.00983935],
       [0.00825817, 0.00933369, 0.00793947, ..., 0.00928062, 0.00797273,
        0.00983821],
       ...,
       [0.01215699, 0.01274062, 0.01216899, ..., 0.01272049, 0.01205027,
        0.01438101],
       [0.01215659, 0.01273626, 0.01217158, ..., 0.01271451, 0.01203102,
        0.01437984],
       [0.0121563 , 0.01272719, 0.01217946, ..., 0.01269072, 0.01202995,
        0.01436187]])

In [4]:
# get actual y values and convert to array 
data_y = data.iloc[:, 58]
data_y.head()
data_y.shape
data_y = data_y.values
data_y

0    0.009279
1    0.009280
2    0.009283
3    0.009287
4    0.009292
Name: MSFT_VWP, dtype: float64

(8642,)

array([0.00927934, 0.00928027, 0.00928309, ..., 0.01271404, 0.01269025,
       0.01270244])

In [5]:
# training parameters
learning_rate = 0.0001
epochs = 100
batch_size = 100
display_step = 20
lambda_value = 0.1

# network parameters
n_inputs = 4
n_hidden = 128
n_outputs = 1
timesteps = 14

In [6]:
def loss(lambda_val, y_actual, y_predicted, weights_dict, norm_order=2):
    
    """
    This function calculates the loss based on MSE and group penalty.
    MSE is calculated using the actual and predicted output values.
    The group penalty is the l2 regularization applied on the weigths.
    
    Inputs:
    lambda_val: value of regularization parameters
    y_actual: tensor of actual output values
    y_predicted: tensor of predicted output values
    weights_dict: dictionary of weights
    norm_order: the order for penality, 1 = l1, 2 = l2 etc. Default is 2.
    
    Output: The calculated loss value
    """
    
    mse = tf.reduce_mean(tf.square(tf.subtract(y_actual, y_predicted)))
    
    penalty_placeholder = tf.get_variable(shape=(1, 1), dtype=tf.float32, initializer=tf.zeros_initializer(), name="penalty_placeholder")
    
    # calculate the norm on each weight matrix
    for key, value in weights_dict.items():
        penalty = tf.norm(value, ord=norm_order)
        penalty_placeholder = tf.add(penalty_placeholder, penalty)
    
    # the combined penalty = mse + lambda * norm(weights)
    loss = tf.add(mse, tf.matmul(lambda_val, penalty_placeholder))
    
    return loss

In [ ]:
with tf.Graph().as_default() as graph:

    X = tf.placeholder("float", [None, timesteps, n_inputs], name="X")
    Y = tf.placeholder("float", [None], name="Y")

    lambda_val = tf.fill((1, 1), lambda_value, name="lambda")

    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, n_outputs]), name="weights_out")
    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_outputs]), name="biases_out")
    }

    weights_dict = data = {k: [] for k in range(n_inputs)}

    # loss and optimizer

    # current data input shape: (batch_size, timesteps, n_input)
    # required shape: 'timesteps' tensors list of shape (batch_size, n_input)
    x = tf.unstack(X, timesteps, 1) # # tensors of shape (batch_size, n_inputs)

    # define lstm cell
    lstm_cell = rnn.BasicLSTMCell(n_hidden)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    logits = tf.matmul(outputs[-1], weights['out']) + biases['out']

    # get the difference weight components
    weights = lstm_cell.get_weights()[0]
    w_i, w_C, w_f, w_o = np.split(weights, 4, axis=1)

    w_xi = w_i[:n_inputs, :]
    w_hi = w_i[n_inputs:, :]

    w_xC = w_C[:n_inputs, :]
    w_hC = w_C[n_inputs:, :]

    w_xf = w_f[:n_inputs, :]
    w_hf = w_f[n_inputs:, :]

    w_xo = w_o[:n_inputs, :]
    w_ho = w_o[n_inputs:, :]

    # concatenate the weight components and transpose so that we have the stocks as columns
    W = np.concatenate((w_xi, w_xC, w_xf, w_xo), axis = 1)
    W_transpose = W.T

    # get the weights for each stock and store in weight_dict
    for i in range(n_inputs):
        weights_dict[i] = W_transpose[:, i]

    prediction = tf.nn.softmax(logits)

    loss_op = loss(lambda_val, Y, logits, weights_dict)

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)

    # evaluation
    accuracy = tf.reduce_mean(tf.square(tf.subtract(Y, logits)))
    
    # initialization
    init = tf.global_variables_initializer()

    # summary
    tf.summary.scalar('loss_op', loss_op[0][0])
    tf.summary.scalar('accuracy', accuracy)
    saver = tf.train.Saver()


with tf.Session(graph=graph) as sess:
    
    # tensorboard graph
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./graphs', sess.graph)

    sess.run(init)

    for epoch in range(epochs):
        for i in range(data_x.shape[0] // batch_size):
            batch_x, batch_y = data_x[i*batch_size:(i+1)*batch_size], data_y[i*batch_size:(i+1)*batch_size]
            batch_x = batch_x.reshape((batch_size, timesteps, n_inputs))
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            
        # loss and accuracy for epoch
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})

        # add summary for epoch
        summary_epoch = sess.run(summary, feed_dict = {X:batch_x, Y:batch_y})
        writer.add_summary(summary_epoch, epoch)

        # display loss and accuracy for each display_step
        if epoch % display_step == 0:
            print("Step: ", str(epoch), "Minibatch Loss: ", loss, ", Training Accuracy: ", acc)   

<tf.Tensor 'loss_op:0' shape=() dtype=string>

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [ ]:
# view the weights
W_abs = abs(W_transpose)
W_abs.sum(axis = 0)

In [ ]:
W_abs[:,0]